In [1]:
#!pip install datefinder

In [2]:
#Importing the required librarie and functions
#a warnings filter is also activated

import pandas as pd
import numpy as np
import datefinder as dfi
import datetime
from pandas import ExcelWriter
import warnings 
from src.cleanfuncs import *
warnings.simplefilter('ignore')

# A. Trade flows (Canary)

In [28]:
#Importing the downloaded dataset and taking a look at it

IC_trade = pd.read_csv ('data/canary_imports_dirty.csv')
IC_trade.head(5)

,Unnamed: 0,Time,Polonia,Chequia,Suiza,Francia,Italia,Bélgica,Dinamarca,Suecia,Irlanda,Países Bajos,Reino Unido,Alemania
0,0,2021 Noviembre (p),"4.473,862","4.698,110","2.220,403","31.485,825","36.338,226","6.008,680","2.178,621","1.574,086","6.561,863","36.734,327","11.720,918","29.201,971"
1,1,2021 Octubre (p),"3.360,605","9.997,751","1.711,822","7.672,856","12.718,099","4.617,084","2.493,726","1.773,129","3.440,667","29.173,793","6.908,026","25.644,328"
2,2,2021 Septiembre (p),"2.619,986","7.827,948","1.331,296","11.305,727","10.842,908","6.359,599","1.133,544","1.540,708","5.547,320","39.631,925","6.700,103","26.015,663"
3,3,2021 Agosto (p),"2.627,623","3.586,898","1.031,052","7.893,321","13.490,680","11.802,737","1.636,533","1.318,356","5.301,273","16.572,673","4.030,151","39.025,092"
4,4,2021 Julio (p),"2.102,901","10.212,904","1.264,930","7.513,558","16.195,542","10.413,309","1.557,271","1.475,441","4.772,880","18.454,604","3.430,143","35.785,879"


In [29]:
#Applying different functions to clean the "Time" column
#and displaying the result

IC_trade.Time = IC_trade.Time.apply(del_years)
IC_trade.Time = IC_trade.Time.apply(del_p)
IC_trade.Time = IC_trade.Time.apply(del_years)
IC_trade.Time = IC_trade.Time.apply(del_spaces)
IC_trade.Time = IC_trade.Time.apply(translate_month)
IC_trade.drop(IC_trade.index[IC_trade['Time'] == 'tbd'], inplace=True)
IC_trade.Time = IC_trade.Time.apply(datematcher)
IC_trade.Time = IC_trade.Time.apply(dateconverter)
IC_trade = IC_trade[IC_trade.Time < datetime.date(2020, 1, 7)]
IC_trade = IC_trade.drop('Unnamed: 0', axis=1)

IC_trade.sample(5)

,Time,Polonia,Chequia,Suiza,Francia,Italia,Bélgica,Dinamarca,Suecia,Irlanda,Países Bajos,Reino Unido,Alemania
93,2014-09-07,"2.935,061","3.840,276","2.111,727","24.029,956","9.913,314","11.750,173","10.262,954","45.280,694","3.267,966","20.462,877","11.284,999","32.491,937"
145,2010-09-07,"1.439,114","1.500,618","2.149,575","9.752,443","25.221,865","13.121,615","2.093,422","2.906,517","5.907,002","47.408,346","18.759,507","25.213,329"
45,2018-05-07,"2.768,404","3.066,499","3.993,820","50.863,321","16.506,455","8.694,127","3.064,628","8.271,721","4.865,223","14.734,677","9.995,599","39.618,161"
79,2015-10-07,"1.786,459","2.924,156","2.128,552","10.032,066","13.354,621","8.488,150","2.458,491","11.290,370","4.734,722","20.651,787","13.783,452","31.970,612"
27,2019-10-07,"3.633,998","3.353,271","1.958,736","13.071,168","35.404,690","5.562,465","3.063,578","1.948,982","5.565,074","28.494,385","17.819,156","50.706,669"


In [31]:
# Checking the presence of null values, no nulls are detected
pd.isnull(IC_trade).sum()

Time            0
Polonia         0
Chequia         0
Suiza           0
Francia         0
Italia          0
Bélgica         0
Dinamarca       0
Suecia          0
Irlanda         0
Países Bajos    0
Reino Unido     0
Alemania        0
dtype: int64

# B. Trade flows (Balearic)

In [10]:
#Importing the downloaded dataset and taking a look at it
BI_trade = pd.read_csv ('data/balearic_imports_dirty.csv')
BI_trade.head(5)

In [12]:
#Applying different functions to clean the "Time" column
#and displaying the result

BI_trade.Time = BI_trade.Time.apply(del_total)
BI_trade.Time = BI_trade.Time.apply(del_years)
BI_trade.drop(BI_trade.index[BI_trade['Time'] == 'tbd'], inplace=True)
BI_trade.Time = BI_trade.Time.apply(del_acumulado)
BI_trade.Time = BI_trade.Time.apply(translate_number)
BI_trade.Time = BI_trade.Time.apply(datematcher)
BI_trade.Time = BI_trade.Time.apply(dateconverter)
BI_trade = BI_trade.drop('Unnamed: 0', axis=1)

BI_trade.sample(5)

In [32]:
# Checking the presence of null values, no nulls are detected
pd.isnull(BI_trade).sum()

Time               0
Polonia            0
República Checa    0
Francia            0
Italia             0
Bélgica            0
Dinamarca          0
Suecia             0
Irlanda            0
Países Bajos       0
Reino Unido        0
Alemania           0
dtype: int64

# C. Tourists in the Canaries

In [14]:
#Countries for which there is detailed data for number of visitors and imports initially
#The sample may be subject to changes 

countries = ['UK', 'HOLANDA', 'IRLANDA', 'SUECIA', 'BELGICA', 'ITALIA', 'FRANCIA', 'REPCHECA', 'POLONIA', 'ALEMANIA', 'SUIZA', 'DINAMARCA']
countries_noger = ['UK', 'HOLANDA', 'IRLANDA', 'SUECIA', 'BELGICA', 'ITALIA', 'FRANCIA', 'REPCHECA', 'POLONIA', 'SUIZA', 'DINAMARCA']

In [15]:
#The following code is employed to create a new sub-dataframe
#for the trade flows data of each country, modify the name of the headers
#and add a new column to keep track of the referredthe country 
#once the dataframes are merged into a new form later on

Polonia_IC = pd.DataFrame(IC_trade[['Time','Polonia']])
Polonia_IC = Polonia_IC.rename({'Polonia': 'Imports'}, axis=1)
Polonia_IC['Country'] = ['PLN' for v in range (0,240)]

República_Checa_IC = pd.DataFrame(IC_trade[['Time','Chequia']])
República_Checa_IC = República_Checa_IC.rename({'Chequia': 'Imports'}, axis=1)
República_Checa_IC['Country'] = ['CZE' for v in range (0,240)]

Francia_IC = pd.DataFrame(IC_trade[['Time','Francia']])
Francia_IC = Francia_IC.rename({'Francia': 'Imports'}, axis=1)
Francia_IC['Country'] = ['FRA' for v in range (0,240)]

Italia_IC = pd.DataFrame(IC_trade[['Time','Italia']])  
Italia_IC = Italia_IC.rename({'Italia': 'Imports'}, axis=1)
Italia_IC['Country'] = ['ITA' for v in range (0,240)]

Bélgica_IC = pd.DataFrame(IC_trade[['Time','Bélgica']])  
Bélgica_IC = Bélgica_IC.rename({'Bélgica': 'Imports'}, axis=1)
Bélgica_IC['Country'] = ['BEL' for v in range (0,240)]

Suecia_IC = pd.DataFrame(IC_trade[['Time','Suecia']])
Suecia_IC = Suecia_IC.rename({'Suecia': 'Imports'}, axis=1)
Suecia_IC['Country'] = ['SWE' for v in range (0,240)]

Irlanda_IC = pd.DataFrame(IC_trade[['Time','Irlanda']])  
Irlanda_IC = Irlanda_IC.rename({'Irlanda': 'Imports'}, axis=1)
Irlanda_IC['Country'] = ['IRL' for v in range (0,240)]

Países_Bajos_IC = pd.DataFrame(IC_trade[['Time','Países Bajos']]) 
Países_Bajos_IC = Países_Bajos_IC.rename({'Países Bajos': 'Imports'}, axis=1)
Países_Bajos_IC['Country'] = ['NLD' for v in range (0,240)]

Reino_Unido_IC = pd.DataFrame(IC_trade[['Time','Reino Unido']]) 
Reino_Unido_IC = Reino_Unido_IC.rename({'Reino Unido': 'Imports'}, axis=1)
Reino_Unido_IC['Country'] = ['GBR' for v in range (0,240)]

Alemania_IC = pd.DataFrame(IC_trade[['Time','Alemania']])
Alemania_IC = Alemania_IC.rename({'Alemania': 'Imports'}, axis=1)
Alemania_IC['Country'] = ['GER' for v in range (0,240)]

Dinamarca_IC = pd.DataFrame(IC_trade[['Time','Dinamarca']])
Dinamarca_IC = Dinamarca_IC.rename({'Dinamarca': 'Imports'}, axis=1)
Dinamarca_IC['Country'] = ['DEN' for v in range (0,240)]

Suiza_IC = pd.DataFrame(IC_trade[['Time','Suiza']])
Suiza_IC = Suiza_IC.rename({'Suiza': 'Imports'}, axis=1)
Suiza_IC['Country'] = ['SWI' for v in range (0,240)]


with ExcelWriter('data/support_tourist.xlsx') as writer: 
    
    Polonia_IC.to_excel(writer, sheet_name='POLONIA')
    República_Checa_IC.to_excel(writer, sheet_name='REPCHECA')
    Francia_IC.to_excel(writer, sheet_name='FRANCIA')
    Italia_IC.to_excel(writer, sheet_name='ITALIA') 
    Bélgica_IC.to_excel(writer, sheet_name='BELGICA') 
    Suecia_IC.to_excel(writer, sheet_name='SUECIA')
    Irlanda_IC.to_excel(writer, sheet_name='IRLANDA')
    Países_Bajos_IC.to_excel(writer, sheet_name='HOLANDA')
    Reino_Unido_IC.to_excel(writer, sheet_name='UK')
    Alemania_IC.to_excel(writer, sheet_name='ALEMANIA') 
    
    Suiza_IC.to_excel(writer, sheet_name='SUIZA')
    Dinamarca_IC.to_excel(writer, sheet_name='DINAMARCA')                             

In [16]:
#MACROLOOP FUNCIONAL (COMENTAR EN INGLÉS...)

for c in countries:
    df_A = pd.read_excel('data/TuristasCI_Short.xls', sheet_name= f'{c}')
    df_B = pd.read_excel('data/support_tourist.xlsx', sheet_name= f'{c}')
    df_A.drop(labels='TOTAL', inplace=True, axis=1)
    df_A.rename(columns={"Unnamed: 0":"Year"}, inplace=True)
    df_A = df_A[df_A.Year > 1999]
    df_A = df_A[df_A.Year < 2020]
    
    df_AT = df_A.T   

    df_AT.columns = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
    df_AT = df_AT.reset_index()
    df_AT.drop(labels='index', inplace=True, axis=1)
    df_AT = df_AT.iloc[1: , :]
    
    df_C = df_AT['1']
    
    for i in range(2,21):
        df_D = df_AT[f'{i}']
        df_C = pd.concat([df_C, df_D])
        
    df_C = df_C.to_frame()
    df_C.columns = ['Column_A']
    print(f'Saving {c}...')
    df_C = df_C.reset_index()
    df_B['Tourist'] = df_C['Column_A']
    df_B = df_B.reset_index()
    df_B.rename(columns={"Unnamed: 0":"Delete"}, inplace=True)
    df_B.drop(labels='Delete', inplace=True, axis=1)
    lista_B = df_B['Tourist'].to_list()
    lista_B.reverse()
    df_B['Tourists'] = lista_B
    df_B.drop(labels='Tourist', inplace=True, axis=1)

    with ExcelWriter(f'data/support_tourist_folder/{c}.xlsx') as writer:
        df_B.to_excel(writer, sheet_name= f'{c}')

Saving UK...
Saving HOLANDA...
Saving IRLANDA...
Saving SUECIA...
Saving BELGICA...
Saving ITALIA...
Saving FRANCIA...
Saving REPCHECA...
Saving POLONIA...
Saving ALEMANIA...
Saving SUIZA...
Saving DINAMARCA...


In [17]:
df_X = pd.read_excel('data/support_tourist_folder/ALEMANIA.xlsx')
for c in countries_noger:
    df_Y = pd.read_excel(f'data/support_tourist_folder/{c}.xlsx')
    df_X = pd.concat([df_X, df_Y])

df_X.rename(columns={"Unnamed: 0":"Delete"}, inplace=True)
df_X.drop(labels='Delete', inplace=True, axis=1)
df_X.rename(columns={"index":"Delete"}, inplace=True)
df_X.drop(labels='Delete', inplace=True, axis=1)
df_X['Year'] = df_X['Time'].dt.year
df_X['Imports'] = df_X['Imports'].apply(punct_num)
df_X["Imports"] = pd.to_numeric(df_X["Imports"], downcast="float")

display(df_X)

,Time,Imports,Country,Tourists,Year
0,2019-12-07,36917.476562,GER,270865,2019
1,2019-11-07,35035.882812,GER,270307,2019
2,2019-10-07,50706.667969,GER,213744,2019
3,2019-09-07,23587.550781,GER,182978,2019
4,2019-08-07,30583.541016,GER,187302,2019
...,...,...,...,...,...
235,2000-05-07,3160.726074,DEN,8718,2000
236,2000-04-07,2636.031982,DEN,21934,2000
237,2000-03-07,3576.811035,DEN,32809,2000
238,2000-02-07,2452.499023,DEN,31644,2000


# D. Tourists in the Balearics

In [21]:
countries_nosui = ['Reino Unido', 'Paises Bajos', 'Belgica', 'Italia', 'Francia', 'Alemania']
countries_noger_nosui = ['Reino Unido', 'Paises Bajos', 'Belgica', 'Italia', 'Francia']
country_sample = ['Belgica','Francia' 'Italia', 'Paises Bajos', 'Reino Unido', 'Alemania', 'Suiza' ,'Rusia']

In [22]:
#The following code is employed to create a new sub-dataframe
#for the trade flows data of each country, modify the name of the headers
#and add a new column to keep track of the referredthe country 
#once the dataframes are merged into a new form later on

Polonia_BI = pd.DataFrame(BI_trade[['Time','Polonia']]).rename({'Polonia': 'Imports'}, axis=1)
Polonia_BI = Polonia_BI.rename({'Polonia': 'Imports'}, axis=1)
Polonia_BI['Country'] = ['PLN' for v in range (0,263)]

República_Checa_BI = pd.DataFrame(BI_trade[['Time','República Checa']])
República_Checa_BI = República_Checa_BI.rename({'Chequia': 'Imports'}, axis=1)
República_Checa_BI['Country'] = ['CZE' for v in range (0,263)]

Francia_BI = pd.DataFrame(BI_trade[['Time','Francia']])
Francia_BI = Francia_BI.rename({'Francia': 'Imports'}, axis=1)
Francia_BI['Country'] = ['FRA' for v in range (0,263)]

Italia_BI = pd.DataFrame(BI_trade[['Time','Italia']])  
Italia_BI = Italia_BI.rename({'Italia': 'Imports'}, axis=1)
Italia_BI['Country'] = ['ITA' for v in range (0,263)]

Bélgica_BI = pd.DataFrame(BI_trade[['Time','Bélgica']])  
Bélgica_BI = Bélgica_BI.rename({'Bélgica': 'Imports'}, axis=1)
Bélgica_BI['Country'] = ['BEL' for v in range (0,263)]

Suecia_BI = pd.DataFrame(BI_trade[['Time','Suecia']])
Suecia_BI = Suecia_BI.rename({'Suecia': 'Imports'}, axis=1)
Suecia_BI['Country'] = ['SWE' for v in range (0,263)]

Irlanda_BI = pd.DataFrame(BI_trade[['Time','Irlanda']])  
Irlanda_BI = Irlanda_BI.rename({'Irlanda': 'Imports'}, axis=1)
Irlanda_BI['Country'] = ['IRL' for v in range (0,263)]

Países_Bajos_BI = pd.DataFrame(BI_trade[['Time','Países Bajos']]) 
Países_Bajos_BI = Países_Bajos_BI.rename({'Países Bajos': 'Imports'}, axis=1)
Países_Bajos_BI['Country'] = ['NLD' for v in range (0,263)]

Reino_Unido_BI = pd.DataFrame(BI_trade[['Time','Reino Unido']]) 
Reino_Unido_BI = Reino_Unido_BI.rename({'Reino Unido': 'Imports'}, axis=1)
Reino_Unido_BI['Country'] = ['GBR' for v in range (0,263)]

Alemania_BI = pd.DataFrame(BI_trade[['Time','Alemania']])
Alemania_BI = Alemania_BI.rename({'Alemania': 'Imports'}, axis=1)
Alemania_BI['Country'] = ['GER' for v in range (0,263)]

Dinamarca_BI = pd.DataFrame(BI_trade[['Time','Dinamarca']])
Dinamarca_BI = Dinamarca_BI.rename({'Dinamarca': 'Imports'}, axis=1)
Dinamarca_BI['Country'] = ['DEN' for v in range (0,263)]


with ExcelWriter('data/support_tourist_2.xlsx') as writer: 
   
    Polonia_BI.to_excel(writer, sheet_name='POLONIA')
    República_Checa_BI.to_excel(writer, sheet_name='REPCHECA')
    Francia_BI.to_excel(writer, sheet_name='Francia')
    Italia_BI.to_excel(writer, sheet_name='Italia') 
    Bélgica_BI.to_excel(writer, sheet_name='Belgica') 
    Suecia_BI.to_excel(writer, sheet_name='SUECIA')
    Irlanda_BI.to_excel(writer, sheet_name='IRLANDA')
    Países_Bajos_BI.to_excel(writer, sheet_name='Paises Bajos')
    Reino_Unido_BI.to_excel(writer, sheet_name='Reino Unido')
    Alemania_BI.to_excel(writer, sheet_name='Alemania') 
   
    Dinamarca_BI.to_excel(writer, sheet_name='DINAMARCA')

In [23]:
df_K = pd.read_csv('data/TuristasBI_Short_C.csv', sep=';')


df_K.rename(columns={"Unnamed: 0":"Time"}, inplace=True)
df_K["Time"] = df_K["Time"].apply(translate_number)
df_K["Time"] = df_K["Time"].apply(datematcher)
df_K = df_K[df_K.Time > datetime.datetime(2005,1,1)]
df_K = df_K[df_K.Time < datetime.datetime(2020,1,1)]

df_K = df_K.replace('...', np.nan)
df_K.drop(labels='Rusia', inplace=True, axis=1)
df_K = df_K.reset_index()
df_K.rename(columns={"index":"Delete"}, inplace=True)
df_K.drop(labels='Delete', inplace=True, axis=1)
 

display(df_K)

df_K.isna().sum()

,Time,Belgica,Francia,Italia,Paises Bajos,Reino Unido,Alemania,Suiza
0,2019-12-07,1846,8395,3419,2933,15391,69008,3878
1,2019-11-07,1975,8608,5582,5557,15415,108237,5966
2,2019-10-07,14214,64842,29927,48883,337509,466520,45825
3,2019-09-07,35823,94693,102521,76201,606611,645933,62384
4,2019-08-07,41162,161094,192856,108556,646813,593696,61803
...,...,...,...,...,...,...,...,...
175,2005-05-07,15555,38240,35180,28198,454937,454274,23962
176,2005-04-07,8110,40556,7547,3532,145653,231734,12940
177,2005-03-07,3605,9634,3869,2133,87277,233349,6693
178,2005-02-07,1722,5258,2107,1000,44718,160791,5363


Time            0
Belgica         0
Francia         0
Italia          0
Paises Bajos    0
Reino Unido     0
Alemania        0
Suiza           0
dtype: int64

In [24]:
#MACROLOOP FUNCIONAL (COMENTAR EN INGLÉS...)

for c in countries_nosui:
    #df_L = pd.read_excel('data/TuristasBI_Short_C.xls', )
    df_L = pd.read_excel('data/support_tourist_2.xlsx', sheet_name= f'{c}')
    df_L = df_L[df_L.Time > datetime.datetime(2005,1,1)]
    df_L = df_L[df_L.Time < datetime.datetime(2020,1,1)]
    df_L = df_L.reset_index()
    df_L.rename(columns={"Unnamed: 0":"Delete"}, inplace=True)
    df_L.drop(labels='Delete', inplace=True, axis=1)
    df_L.rename(columns={"index":"Delete"}, inplace=True)
    df_L.drop(labels='Delete', inplace=True, axis=1)
    df_L['Tourist'] = df_K[f'{c}']
    df_L['Imports'] = df_L['Imports'].apply(punct_num)
    

    with ExcelWriter(f'data/support_tourist_folder_2/{c}.xlsx') as writer:
        df_L.to_excel(writer, sheet_name= f'{c}')

In [25]:
df_M = pd.read_excel('data/support_tourist_folder_2/Alemania.xlsx')
for c in countries_noger_nosui:
    df_N = pd.read_excel(f'data/support_tourist_folder_2/{c}.xlsx')
    df_M = pd.concat([df_M, df_N])

df_M['Year'] = df_M['Time'].dt.year


display(df_M)

,Unnamed: 0,Time,Imports,Country,Tourist,Year
0,0,2019-12-07,24961994,GER,69008,2019
1,1,2019-11-07,20791329,GER,108237,2019
2,2,2019-10-07,21244446,GER,466520,2019
3,3,2019-09-07,22357741,GER,645933,2019
4,4,2019-08-07,20255555,GER,593696,2019
...,...,...,...,...,...,...
175,175,2005-05-07,34803435,FRA,38240,2005
176,176,2005-04-07,4768458,FRA,40556,2005
177,177,2005-03-07,7480924,FRA,9634,2005
178,178,2005-02-07,4274270,FRA,5258,2005


# C. Foreign citizens in the Canaries and the Balearic islands

# Z. Testbench

In [ ]:
#dataframe = pd.DataFrame(df_X["Tourists","Year"].groupby(df_X["Country", ]).sum())
df_X_yearly = df_X.groupby(["Year", "Country"]).sum()

display(df_X_yearly)


In [ ]:
df_X_yearly = df_X.groupby(["Year"]).sum()

display(df_X_yearly)

In [ ]:
df_X_yearly.to_dict()